In [30]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, optimizers, callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16
import numpy as np
import os
import matplotlib.pyplot as plt
from datasets import load_dataset

In [31]:
class FacialDetectionModel:
    def __init__(self, input_shape=(512,), num_classes=10, log_dir='./logs'):
        """
        Initialize the facial detection model with TensorBoard logging
        
        Parameters:
        - input_shape: Shape of the embedding input
        - num_classes: Number of unique labels in the dataset
        - log_dir: Directory for TensorBoard logs
        """
        self.model = None
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.log_dir = log_dir
        
        # Ensure log directory exists
        os.makedirs(log_dir, exist_ok=True)
    
    def prepare_dataset(self, dataset):
        """
        Prepare dataset for training
        
        Returns:
        - X_train, y_train, X_test, y_test
        """
        # Convert embeddings and labels to numpy arrays
        X_train = np.array(dataset['train']['embedding'])
        y_train = np.array(dataset['train']['label'])
        X_test = np.array(dataset['test']['embedding'])
        y_test = np.array(dataset['test']['label'])
        
        # One-hot encode labels
        y_train = tf.keras.utils.to_categorical(y_train, num_classes=self.num_classes)
        y_test = tf.keras.utils.to_categorical(y_test, num_classes=self.num_classes)
        
        return X_train, y_train, X_test, y_test
    
    def build_model(self,learning_rate=1e-3):
        """
        Build a neural network for classification
        """
        self.model = models.Sequential([
            layers.Input(shape=self.input_shape),
            # First dense layer with L2 regularization and dropout
            layers.Dense(264, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
            layers.Dropout(0.4),

            # Second dense layer
            layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
            layers.Dropout(0.4),

            # Third dense layer
            layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
            layers.Dropout(0.3),
            layers.Dense(self.num_classes, activation='softmax')
        ])
        
        # Advanced optimizer with adaptive learning rate
        optimizer = optimizers.Adam(
            learning_rate=learning_rate, 
            clipnorm=1.0  # Gradient clipping to prevent exploding gradients
        )

        # Compile with advanced metrics
        self.model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=[
                'accuracy',
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                tf.keras.metrics.AUC()
            ]
        )
        print(self.model.summary())
    
    def train(self, dataset, epochs=100, batch_size=8):
        """
        Train the model with TensorBoard logging
        
        Returns:
        - Training history
        """
        # Prepare dataset
        X_train, y_train, X_test, y_test = self.prepare_dataset(dataset)
        
        # Build model if not already built
        if self.model is None:
            self.build_model()
        
        # Callbacks for advanced training
        callbacks_list = [
            
            # Early stopping to prevent overfitting
            callbacks.EarlyStopping(
                monitor='val_loss', 
                patience=10, 
                restore_best_weights=True
            ), 
            # Reduce learning rate when plateau occurs
            callbacks.ReduceLROnPlateau(
                monitor='val_loss', 
                factor=0.5, 
                patience=5, 
                min_lr=1e-5
            ),
            
            # TensorBoard for visualization
            callbacks.TensorBoard(
                log_dir=self.log_dir, 
                histogram_freq=1,
                profile_batch=0
            ),
            
            # Model checkpointing
            callbacks.ModelCheckpoint(
                filepath=os.path.join(self.log_dir, 'best_model.keras'),
                monitor='val_accuracy',
                save_best_only=True
            )
        ]
        
        
        # Train the model with callbacks
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks_list,
            verbose=1
        )
        
        return history
    
    def plot_training_history(self, history):
        """
        Create comprehensive plots of training history
        
        Parameters:
        - history: Model training history
        """
        # Create a figure with multiple subplots
        plt.figure(figsize=(10, 5))
        
        # Plot training & validation accuracy values
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='lower right')
        
        # Plot training & validation loss values
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper right')
        
        plt.tight_layout()
        plt.savefig('training_history.png')
        plt.close()
    
    def run_tensorboard(self):
        """
        Start TensorBoard server
        
        Note: Run this in a separate terminal or notebook cell
        """
        import subprocess
        
        # Start TensorBoard
        tensorboard_process = subprocess.Popen(
            ['tensorboard', '--logdir', self.log_dir]
        )
        print(f"TensorBoard running. Open http://localhost:6006 in your browser.")
        return tensorboard_process


In [32]:
 # Assuming you've already loaded your dataset
dataset = load_dataset("Tarakeshwaran/sampleface30-Dataset")  # Your dataset loading code

README.md:   0%|          | 0.00/418 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/9.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3586 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1538 [00:00<?, ? examples/s]

In [33]:
dataset

DatasetDict({
    train: Dataset({
        features: ['embedding', 'label'],
        num_rows: 3586
    })
    test: Dataset({
        features: ['embedding', 'label'],
        num_rows: 1538
    })
})

In [34]:
# Initialize and train the model
facial_model = FacialDetectionModel(
    input_shape=(len(dataset['train'][0]['embedding']),),
    num_classes=len(set(dataset['train']['label']))
)

# Train the model
history = facial_model.train(dataset)

# Plot training history
facial_model.plot_training_history(history)
"""
# Convert to TFLite
facial_model.convert_to_tflite()

# Optional: Evaluate the model
facial_model.evaluate(dataset)"""

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 264)               338184    
                                                                 
 dropout_23 (Dropout)        (None, 264)               0         
                                                                 
 dense_41 (Dense)            (None, 128)               33920     
                                                                 
 dropout_24 (Dropout)        (None, 128)               0         
                                                                 
 dense_42 (Dense)            (None, 64)                8256      
                                                                 
 dropout_25 (Dropout)        (None, 64)                0         
                                                                 
 dense_43 (Dense)            (None, 31)               

'\n# Convert to TFLite\nfacial_model.convert_to_tflite()\n\n# Optional: Evaluate the model\nfacial_model.evaluate(dataset)'